In [9]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="white")
import os
import plotly.express as px
from plotly.subplots import make_subplots
from plotly import graph_objects as go
import warnings
warnings.filterwarnings("ignore")

In [10]:
file_str = ['ground_truth_{}','particle_uncertainty_truth_{}']
params = ["0.000000.csv","0.050000.csv","0.100000.csv","0.150000.csv","0.200000.csv","0.250000.csv","0.300000.csv","0.350000.csv","0.400000.csv","0.450000.csv","0.500000.csv",
"0.550000.csv","0.600000.csv","0.650000.csv","0.700000.csv","0.750000.csv","0.800000.csv","0.850000.csv","0.900000.csv","0.950000.csv"]


original_dfs = []

for st in file_str:
    for val in params:
        
        fh = st.format(val)
        df = pd.read_csv(fh,header=None)
        df =df.rename(columns={0:'time_str',1:'time_taken_lec',2:'time_taken_safety_controller',3:'area'})
        df['parameter_uncertainty']  = float(val.replace(".csv",""))
        df['mode'] = st[:-3]
        df = df.tail(30)
        original_dfs.append(df)
        
dfs = pd.concat(original_dfs)
        
        


In [11]:
dfs.groupby(['mode','parameter_uncertainty']).mean()

time_taken_lec  \
mode                       parameter_uncertainty                   
ground_truth               0.00                         0.801463   
                           0.05                         0.673913   
                           0.10                         0.785917   
                           0.15                         0.763021   
                           0.20                         0.627402   
                           0.25                         0.527946   
                           0.30                         0.449302   
                           0.35                         0.388647   
                           0.40                         0.362265   
                           0.45                              NaN   
                           0.50                         0.265372   
                           0.55                         0.260849   
                           0.60                         0.235175   
                           0.65                         0.241552   
                           0.70                         0.219918   
                           0.75                         0.214259   
                           0.80                         0.216294   
                           0.85                              NaN   
                           0.90                              NaN   
                           0.95                              NaN   
particle_uncertainty_truth 0.00                         0.907722   
                           0.05                         0.867603   
                           0.10                         0.830767   
                           0.15                         0.816359   
                           0.20                         0.765766   
                           0.25                         0.766550   
                           0.30                         0.789832   
                           0.35                         0.714585   
                           0.40                         0.781459   
                           0.45                         0.044373   
                           0.50                         0.603402   
                           0.55                         0.036212   
                           0.60                         0.017570   
                           0.65                         0.027010   
                           0.70                         0.023107   
                           0.75                         0.012189   
                           0.80                         0.006519   
                           0.85                         0.016673   
                           0.90                         0.006721   
                           0.95                         0.003859   

                                                  time_taken_safety_controller  \
mode                       parameter_uncertainty                                 
ground_truth               0.00                                       0.198537   
                           0.05                                       0.326087   
                           0.10                                       0.214083   
                           0.15                                       0.236979   
                           0.20                                       0.372598   
                           0.25                                       0.472054   
                           0.30                                       0.550698   
                           0.35                                       0.611353   
                           0.40                                       0.637735   
                           0.45                                            NaN   
                           0.50                                       0.734628   
                           0.55                                       0.739151   
                           0.60              